# Imports

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder

import pickle

import psycopg2

/home/johnrivera/.virtualenvs/ds_reddit_build-L6udXIR8/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# Read in data

In [6]:
with open("../database/secrets", "r") as file:
    secrets = [i.strip('\n') for i in file.readlines()]


def conn_curs():
    """
    makes a connection to the database dont worry these are dummy keys
    """

    connection = psycopg2.connect(dbname=secrets[4], user=secrets[4],
                                  password=secrets[5], host=secrets[6])
    cursor = connection.cursor()
    return connection, cursor

In [7]:
conn, curs = conn_curs()
df = pd.read_sql("SELECT * FROM posts", conn)

# Inspect

In [8]:
df.head()

,id,text,subreddit
0,1,Thousand Year Blood War Arc Anime Adaptation M...,bleach
1,2,Burn The Witch - Chapter 4 Discussion Thread #...,bleach
2,3,Let the journey begin.,bleach
3,4,"Since Ichigos an English Literature major, thi...",bleach
4,5,I just made these Ulquiorra customs for a clie...,bleach


In [9]:
df.shape

(10782, 3)

In [10]:
df.drop('id', axis=1, inplace=True)

In [11]:
df.subreddit.value_counts()

DecidingToBeBetter    300
legaladvice           300
findapath             300
WouldYouRather        300
bleach                300
getting_over_it       300
immigration           300
ShingekiNoKyojin      300
MLPLounge             300
Gundam                300
LifeProTips           300
Gunpla                300
bettafish             300
resumes               300
leopardgeckos         300
BackYardChickens      300
GiftIdeas             300
cats                  300
deathnote             300
nosurf                300
araragi               300
RBI                   300
BokuNoHeroAcademia    300
CaptainTsubasaDT      300
ballpython            300
anime                 300
ferrets               300
christmas             300
Bedbugs               300
DDLC                  300
manga                 300
ask                   300
LegalAdviceUK         300
selfhelp              300
TooAfraidToAsk        300
GetMotivated          220
whatisthisthing        62
Name: subreddit, dtype: int64

# Model

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df.text, df.subreddit, test_size=0.33, random_state=42)

In [13]:
log_pipe = make_pipeline(TfidfVectorizer(max_df=.95,  min_df=80), LogisticRegression(random_state=42, n_jobs=-1))
log_pipe.fit(X_train, y_train);

In [14]:
print(f"Log train accuracy: {log_pipe.score(X_train, y_train)}")
print(f"Log val accuracy: {log_pipe.score(X_test, y_test)}")

Log train accuracy: 0.6422539111172643
Log val accuracy: 0.46024164091036807


In [15]:
forest_pipe = make_pipeline(TfidfVectorizer(max_df=.95, min_df=80), LogisticRegression(random_state=42, n_jobs=-1))
forest_pipe.fit(X_train, y_train);

In [16]:
print(f"Forest train accuracy: {forest_pipe.score(X_train, y_train)}")
print(f"Forest val accuracy: {forest_pipe.score(X_test, y_test)}")

Forest train accuracy: 0.6422539111172643
Forest val accuracy: 0.46024164091036807


In [17]:
sgd_pipe = make_pipeline(TfidfVectorizer(max_df=.95, min_df=80), SGDClassifier(n_jobs=-1))
sgd_pipe.fit(X_train, y_train);

In [18]:
print(f"SGD train accuracy: {sgd_pipe.score(X_train, y_train)}")
print(f"SGD val accuracy: {sgd_pipe.score(X_test, y_test)}")

SGD train accuracy: 0.7404125709538972
SGD val accuracy: 0.4459117729699354


# Hyperparameter Tuning

In [19]:
sgd_tune = make_pipeline(TfidfVectorizer(), SGDClassifier(n_jobs=-1))

params = {
    'sgdclassifier__loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
    'sgdclassifier__learning_rate': ["constant","optimal","invscaling","adaptive"],
    'sgdclassifier__eta0': [.001, .0001, .01],
    'sgdclassifier__early_stopping': [True, False],
    'sgdclassifier__validation_fraction': [.1, .2, .3],
    'tfidfvectorizer__min_df': [30, 50, 80, 100, .1],
    'tfidfvectorizer__max_df': [.95, .9, .97]
}

search = RandomizedSearchCV(sgd_tune, params, random_state=42, cv=3, n_jobs=-1, n_iter=150, verbose=1)

search.fit(X_train, y_train);

Fitting 3 folds for each of 150 candidates, totalling 450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   25.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:  5.6min finished


In [22]:
search.best_params_

{'tfidfvectorizer__min_df': 30,
 'tfidfvectorizer__max_df': 0.97,
 'sgdclassifier__validation_fraction': 0.2,
 'sgdclassifier__loss': 'squared_hinge',
 'sgdclassifier__learning_rate': 'constant',
 'sgdclassifier__eta0': 0.01,
 'sgdclassifier__early_stopping': False}

In [23]:
search.best_score_

0.5236060521154385

In [24]:
# {'tfidfvectorizer__min_df': 30,
#  'tfidfvectorizer__max_df': 0.97,
#  'sgdclassifier__validation_fraction': 0.2,
#  'sgdclassifier__loss': 'squared_hinge',
#  'sgdclassifier__learning_rate': 'constant',
#  'sgdclassifier__eta0': 0.01,
#  'sgdclassifier__early_stopping': False}
# 0.6216751552287852

# Now forest

In [28]:
forest_tune = make_pipeline(TfidfVectorizer(), RandomForestClassifier(random_state=42, n_jobs=-1))

params = {
    'randomforestclassifier__n_estimators': range(100, 501, 100),
    'randomforestclassifier__criterion': ['gini', 'entropy'],
    'randomforestclassifier__max_depth': [None, 5, 10, 40, 100, 200],
    'randomforestclassifier__min_samples_split': range(2, 51, 2),
    'randomforestclassifier__min_samples_leaf': range(1, 51, 2),
    'randomforestclassifier__max_features': ['auto', 'sqrt', 'log2'],
    
    'tfidfvectorizer__min_df': [30, 50, 80, 100, .1],
    'tfidfvectorizer__max_df': [.95, .9, .97]
}

forest_search = RandomizedSearchCV(forest_tune, params, random_state=42, cv=3, n_jobs=-1, n_iter=150, verbose=1)

forest_search.fit(X_train, y_train);

Fitting 3 folds for each of 150 candidates, totalling 450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 11.0min
[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed: 11.5min finished


In [32]:
forest_search.best_params_

{'tfidfvectorizer__min_df': 30,
 'tfidfvectorizer__max_df': 0.97,
 'randomforestclassifier__n_estimators': 100,
 'randomforestclassifier__min_samples_split': 10,
 'randomforestclassifier__min_samples_leaf': 3,
 'randomforestclassifier__max_features': 'auto',
 'randomforestclassifier__max_depth': 100,
 'randomforestclassifier__criterion': 'gini'}

In [33]:
forest_search.best_score_

0.47958054005459344

# MORE REDDITS NUKED OUR SCORES

# TRYING A NET BECAUSE SCORES ARE DEPRESSING

In [34]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, Adagrad, SGD, Ftrl, RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

import os
import datetime

In [35]:
le = LabelEncoder()
le.fit(df.subreddit.unique())

LabelEncoder()

In [36]:
vect = TfidfVectorizer(max_df=.95, min_df=80, stop_words='english')
vect.fit(X_train)

TfidfVectorizer(max_df=0.95, min_df=80, stop_words='english')

In [37]:
logdir = os.path.join("logs", "Sigmoid + Dropout")
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)
stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5)

model = Sequential()
model.add(Dense(256, activation='sigmoid', input_dim=len(vect.get_feature_names())))
model.add(Dropout(.1))
model.add(Dense(128, activation='sigmoid'))
model.add(Dropout(.1))
model.add(Dense(128, activation='sigmoid'))
model.add(Dropout(.1))
model.add(Dense(df.subreddit.nunique(), activation='softmax'))

model.compile(optimizer=Adam(.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [38]:
model.fit(np.array(vect.transform(X_train).todense()),le.transform(y_train),
          validation_data=(np.array(vect.transform(X_test).todense()), le.transform(y_test)),
          batch_size=32, epochs=200, callbacks=[tensorboard_callback, stop])

Epoch 1/200
  1/226 [..............................] - ETA: 0s - loss: 3.7475 - accuracy: 0.0000e+00WARNING:tensorflow:From /home/johnrivera/.virtualenvs/ds_reddit_build-L6udXIR8/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
226/226 [==============================] - 1s 3ms/step - loss: 3.6321 - accuracy: 0.0295 - val_loss: 3.6170 - val_accuracy: 0.0216
Epoch 2/200
226/226 [==============================] - 0s 2ms/step - loss: 3.6056 - accuracy: 0.0323 - val_loss: 3.5937 - val_accuracy: 0.0275
Epoch 3/200
226/226 [==============================] - 0s 2ms/step - loss: 3.5410 - accuracy: 0.0602 - val_loss: 3.3934 - val_accuracy: 0.0840
Epoch 4/200
226/226 [==============================] - 0s 2ms/step - loss: 3.2340 - accuracy: 0.0957 - val_loss: 3.0873 - val_accuracy: 0.1233
Epoch 5/200
226/22

In [39]:
%load_ext tensorboard
%tensorboard --logdir logs

# Tring random search on net

In [42]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [43]:
def create_model(hidden_layers:int, largest_hidden_nueron:int, activation:str, out_activation, optimizer, learning_rate:float):
    # create model
    model = Sequential()
    # first layer is special so make out of for loop
    x = [int(i) for i in np.linspace(64, largest_hidden_nueron, hidden_layers//2)]
    y = x + list(reversed(x))
    model.add(Dense(y[0], input_dim=len(vect.get_feature_names()), activation=activation))
    if hidden_layers > 1:
        for i in range(1, hidden_layers - 1):
            model.add(Dense(y[i], activation=activation))
    model.add(Dense(df.subreddit.nunique(), activation=out_activation))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer(learning_rate), metrics=['accuracy'])
    return model

In [44]:
model = KerasClassifier(build_fn=create_model, verbose=1)

In [45]:
param_grid = {'batch_size': (16,32,64,512),
              'epochs': [100],
              'hidden_layers': (1,3,5,8),
              'largest_hidden_nueron': [512,256,128],
              'activation': ('relu', 'sigmoid', 'tanh'),
              'out_activation': ['softmax'],
              'optimizer': (Adam, Adagrad, SGD, Ftrl, RMSprop),
              'learning_rate': [.001]#tuple(np.linspace(.001, .01, 5))
             }

nueral_search = RandomizedSearchCV(estimator=model,param_distributions=param_grid,n_jobs=-1,cv=3,random_state=42,verbose=1,n_iter=20)
nueral_search.fit(np.array(vect.transform(X_train).todense()),le.transform(y_train),
                  validation_data=(np.array(vect.transform(X_test).todense()), le.transform(y_test)),
                  epochs=32)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  7.4min finished


Epoch 1/32
452/452 [==============================] - 1s 2ms/step - loss: 3.6072 - accuracy: 0.0330 - val_loss: 3.5904 - val_accuracy: 0.0306
Epoch 2/32
452/452 [==============================] - 1s 2ms/step - loss: 3.5603 - accuracy: 0.0605 - val_loss: 3.5399 - val_accuracy: 0.0596
Epoch 3/32
452/452 [==============================] - 1s 2ms/step - loss: 3.4455 - accuracy: 0.1177 - val_loss: 3.3641 - val_accuracy: 0.1326
Epoch 4/32
452/452 [==============================] - 1s 2ms/step - loss: 3.2226 - accuracy: 0.1937 - val_loss: 3.1230 - val_accuracy: 0.2369
Epoch 5/32
452/452 [==============================] - 1s 2ms/step - loss: 2.9957 - accuracy: 0.2572 - val_loss: 2.9092 - val_accuracy: 0.2574
Epoch 6/32
452/452 [==============================] - 1s 2ms/step - loss: 2.7721 - accuracy: 0.2990 - val_loss: 2.7018 - val_accuracy: 0.2900
Epoch 7/32
452/452 [==============================] - 1s 2ms/step - loss: 2.5586 - accuracy: 0.3361 - val_loss: 2.5212 - val_accuracy: 0.3189
Epoch 

RandomizedSearchCV(cv=3,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f5e1c472dd0>,
                   n_iter=20, n_jobs=-1,
                   param_distributions={'activation': ('relu', 'sigmoid',
                                                       'tanh'),
                                        'batch_size': (16, 32, 64, 512),
                                        'epochs': [100],
                                        'hidden_layers': (1, 3, 5, 8),
                                        'largest_hidden_nueron': [512, 256,
                                                                  128],
                                        'learning_rate': [0.001],
                                        'optimizer': (<class...sorflow.python.keras.optimizer_v2.adam.Adam'>,
                                                      <class 'tensorflow.python.keras.optimizer_v2.adagrad.Adagrad'>,
                                  

In [46]:
nueral_search.best_params_

{'out_activation': 'softmax',
 'optimizer': tensorflow.python.keras.optimizer_v2.rmsprop.RMSprop,
 'learning_rate': 0.001,
 'largest_hidden_nueron': 512,
 'hidden_layers': 3,
 'epochs': 100,
 'batch_size': 16,
 'activation': 'sigmoid'}

In [167]:
# logdir = os.path.join("logs", "Best from search")
# tensorboard_callback = TensorBoard(logdir, histogram_freq=1)
# stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10)

# nn = Sequential([
#     Dense(64, activation='sigmoid', input_dim=len(vect.get_feature_names())),
#     Dense(128, activation='sigmoid'),
#     Dense(128, activation='sigmoid'),
#     Dense(64, activation='sigmoid')
# ])
# nn.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(.001), metrics=['accuracy'])

# nn.fit(np.array(vect.transform(X_train).todense()),le.transform(y_train),
#           validation_data=(np.array(vect.transform(X_test).todense()), le.transform(y_test)),
#           batch_size=32, epochs=100)

# THe best model for my data set ended up being the SDGC Classifier